Long-term international migration 2.05, Occupation

In [1]:
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/' \
                  'internationalmigration/datasets/longterminternationalmigrationusualoccupationpriortomigrationtable205')
scraper

## Long-term international migration 2.05, usual occupation prior to migration, UK and England and Wales

Regular job of migrants entering or leaving UK. Estimates of Long-Term International Migration, annual table.

### Distributions

1. Long-term international migration 2.05, usual occupation prior to migration, UK and England and Wales ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/internationalmigration/datasets/longterminternationalmigrationusualoccupationpriortomigrationtable205/current/2.05ltimusualoccupationpriortomigration1991to2016.xls))


In [2]:
tab = next(t for t in scraper.distribution().as_databaker() if t.name == 'Table 2.05')

In [3]:
observations = tab.excel_ref("B14").expand(RIGHT).filter(contains_string("Estimate")).expand(DOWN).is_not_blank().is_not_whitespace()
observations

{<G31 101.0>, <I154 -53.0>, <B79 -13.0>, <M22 37.0>, <G60 -89.0>, <G36 138.0>, <E67 -138.0>, <E101 62.0>, <E63 -146.0>, <B99 177.0>, <M154 -31.0>, <G46 'No'>, <P14 'Estimate'>, <M138 -42.0>, <K170 19.0>, <B109 304.0>, <G128 106.0>, <G28 90.0>, <G86 12.0>, <I109 54.0>, <K142 -23.0>, <G152 -82.0>, <K92 5.0>, <E58 -143.0>, <G169 0.0>, <K111 39.0>, <G22 51.0>, <G62 -78.0>, <B191 309.0>, <M124 47.0>, <G81 9.0>, <E114 97.0>, <M158 -11.0>, <K55 -33.0>, <K59 -36.0>, <G113 54.0>, <B29 479.0>, <E123 175.0>, <M83 -10.0>, <K132 39.0>, <I110 45.0>, <G184 -29.0>, <G65 -104.0>, <B111 249.0>, <P91 245.0>, <M27 41.0>, <E185 20.0>, <G79 -4.0>, <I54 -42.0>, <M186 13.0>, <K151 -45.0>, <B68 -339.0>, <G155 -147.0>, <G53 -50.0>, <I33 161.0>, <M155 -21.0>, <K65 -29.0>, <I102 101.0>, <B34 567.0>, <E50 -102.0>, <G131 110.0>, <M21 36.0>, <E82 22.0>, <I23 60.0>, <B172 62.0>, <G126 118.0>, <B190 298.0>, <K75 'No'>, <G140 -50.0>, <K127 51.0>, <K57 -26.0>, <M33 43.0>, <P94 233.0>, <B28 454.0>, <K43 41.0>, <I121 138.

In [4]:
CI = observations.shift(RIGHT)
CI

{<L153 12.0>, <N26 12.0>, <F164 ''>, <N93 15.0>, <F179 23.0>, <J178 19.0>, <C138 21.0>, <C101 43.0>, <L185 10.0>, <N67 5.0>, <C171 28.0>, <H73 13.0>, <C147 26.0>, <C60 29.0>, <H81 16.0>, <J153 11.0>, <J101 25.0>, <J122 20.0>, <F29 23.0>, <J185 15.0>, <H33 22.0>, <L180 16.0>, <H57 13.0>, <F46 ''>, <F161 12.0>, <N150 9.0>, <H138 9.0>, <F138 12.0>, <C41 27.0>, <H84 14.0>, <F122 22.0>, <L68 4.0>, <J33 21.0>, <F83 20.0>, <J41 16.0>, <L191 8.0>, <F125 19.0>, <L14 '+/-CI'>, <N95 11.0>, <N110 8.0>, <C32 33.0>, <J144 9.0>, <H23 11.0>, <C59 25.0>, <H31 16.0>, <C55 24.0>, <H82 14.0>, <L50 5.0>, <N191 9.0>, <J97 20.0>, <F109 10.0>, <C184 54.0>, <N36 10.0>, <H71 11.0>, <L82 8.0>, <J169 12.0>, <L33 10.0>, <H40 14.0>, <L125 14.0>, <L150 8.0>, <H174 18.0>, <N101 9.0>, <F52 10.0>, <H179 26.0>, <C146 24.0>, <N157 5.0>, <J86 20.0>, <H92 28.0>, <F26 13.0>, <J34 18.0>, <N88 11.0>, <J35 20.0>, <C150 31.0>, <L94 16.0>, <J57 10.0>, <H148 11.0>, <C45 34.0>, <L37 12.0>, <L178 12.0>, <F74 13.0>, <J73 9.0>, <H149

In [5]:
observations = observations - tab.excel_ref('B').expand(DOWN).expand(RIGHT).by_index([46,75,104,135,164,193])
observations

{<G31 101.0>, <I154 -53.0>, <B79 -13.0>, <M22 37.0>, <G60 -89.0>, <G36 138.0>, <E67 -138.0>, <E101 62.0>, <E63 -146.0>, <B99 177.0>, <M154 -31.0>, <G46 'No'>, <P14 'Estimate'>, <M138 -42.0>, <K170 19.0>, <B109 304.0>, <G128 106.0>, <G28 90.0>, <G86 12.0>, <I109 54.0>, <K142 -23.0>, <G152 -82.0>, <K92 5.0>, <E58 -143.0>, <G169 0.0>, <K111 39.0>, <G22 51.0>, <G62 -78.0>, <B191 309.0>, <M124 47.0>, <G81 9.0>, <E114 97.0>, <M158 -11.0>, <K55 -33.0>, <K59 -36.0>, <G113 54.0>, <B29 479.0>, <E123 175.0>, <M83 -10.0>, <K132 39.0>, <I110 45.0>, <G184 -29.0>, <G65 -104.0>, <B111 249.0>, <P91 245.0>, <M27 41.0>, <E185 20.0>, <G79 -4.0>, <I54 -42.0>, <M186 13.0>, <K151 -45.0>, <B68 -339.0>, <G155 -147.0>, <G53 -50.0>, <I33 161.0>, <M155 -21.0>, <K65 -29.0>, <I102 101.0>, <B34 567.0>, <E50 -102.0>, <G131 110.0>, <M21 36.0>, <E82 22.0>, <I23 60.0>, <B172 62.0>, <G126 118.0>, <B190 298.0>, <K75 'No'>, <G140 -50.0>, <K127 51.0>, <K57 -26.0>, <M33 43.0>, <P94 233.0>, <B28 454.0>, <K43 41.0>, <I121 138.

In [6]:
Occupation = tab.excel_ref('B12').expand(RIGHT).is_not_blank()
Occupation

{<K12 'Other adults3'>, <M12 'Children4'>, <E12 'Professional and managerial2'>, <G12 'Manual and clerical2'>, <P12 'All persons'>, <I12 'Students2'>, <B12 'All persons'>}

In [7]:
Geography = tab.excel_ref('A').expand(DOWN).by_index([16,105])
Geography

{<A105 'England and Wales'>, <A16 'United Kingdom'>}

In [8]:
Revision = tab.excel_ref('P13').expand(RIGHT).is_not_blank()
Revision

{<P13 'Original Estimates1'>}

In [9]:
Flow = tab.excel_ref('A').expand(DOWN).by_index([18,47,76,107,136,165])
Flow

{<A47 'Outflow'>, <A107 'Inflow'>, <A18 'Inflow'>, <A76 'Balance'>, <A165 'Balance'>, <A136 'Outflow'>}

In [10]:
Year = tab.excel_ref('A20').expand(DOWN) - Geography - Flow - tab.excel_ref('A').expand(DOWN).by_index([46,75,104,135,164,193]) 
Year

{<A35 2006.0>, <A183 2007.0>, <A154 2007.0>, <A109 1991.0>, <A168 1992.0>, <A122 2004.0>, <A55 1997.0>, <A148 2001.0>, <A83 1996.0>, <A67 2009.0>, <A166 ''>, <A173 1997.0>, <A34 2005.0>, <A128 2010.0>, <A151 2004.0>, <A29 2000.0>, <A176 2000.0>, <A126 2008.0>, <A82 1995.0>, <A98 2011.0>, <A24 1995.0>, <A40 2011.0>, <A203 '3   "Other adults" are migrants aged 16 and over who did not state their occupation prior to migration or who stated that they were retired, unoccupied or a houseperson.'>, <A119 2001.0>, <A23 1994.0>, <A204 '4   All migrants under 16 years of age are grouped as "Children" regardless of any occupation prior to migration.'>, <A22 1993.0>, <A38 2009.0>, <A124 2006.0>, <A25 1996.0>, <A57 1999.0>, <A160 2013.0>, <A71 2013.0>, <A129 2011.0>, <A99 2012.0>, <A20 1991.0>, <A137 ''>, <A153 2006.0>, <A33 2004.0>, <A91 2004.0>, <A77 ''>, <A211 'Published on 30 November 2017 by the Office for National Statistics. Email: migstatsunit@ons.gsi.gov.uk'>, <A94 2007.0>, <A93 2006.0>, <

In [11]:
Tag = tab.excel_ref('B14').expand(RIGHT).is_not_blank()
Tag

{<N14 '+/-CI'>, <K14 'Estimate'>, <H14 '+/-CI'>, <E14 'Estimate'>, <C14 '+/-CI'>, <M14 'Estimate'>, <J14 '+/-CI'>, <G14 'Estimate'>, <Q14 '+/-CI'>, <B14 'Estimate'>, <L14 '+/-CI'>, <I14 'Estimate'>, <F14 '+/-CI'>, <P14 'Estimate'>}

In [12]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,LEFT),
            HDim(Geography,'Geography',CLOSEST,ABOVE),
            HDim(Occupation,'Occupation',CLOSEST,LEFT),
            HDim(Flow,'Flow',CLOSEST,ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People (thousands)'),
            HDim(Tag, 'Tag',DIRECTLY,ABOVE),
            HDim(Revision, 'Revision',DIRECTLY,ABOVE),
            HDim(CI,'CI',DIRECTLY,RIGHT)
            ]

In [13]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
savepreviewhtml(c1)

In [14]:
new_table = c1.topandas()
new_table

,OBS,DATAMARKER,Year,Geography,Occupation,Flow,Measure Type,Unit,Tag,Revision,CI
0,,Estimate,None,None,All persons,None,Count,People (thousands),Estimate,None,+/-CI
1,,Estimate,None,None,Professional and managerial2,None,Count,People (thousands),Estimate,None,+/-CI
2,,Estimate,None,None,Manual and clerical2,None,Count,People (thousands),Estimate,None,+/-CI
3,,Estimate,None,None,Students2,None,Count,People (thousands),Estimate,None,+/-CI
4,,Estimate,None,None,Other adults3,None,Count,People (thousands),Estimate,None,+/-CI
5,,Estimate,None,None,Children4,None,Count,People (thousands),Estimate,None,+/-CI
6,,Estimate,None,None,All persons,None,Count,People (thousands),Estimate,Original Estimates1,+/-CI
7,329,NaN,1991.0,United Kingdom,All persons,Inflow,Count,People (thousands),Estimate,None,23.0
8,94,NaN,1991.0,United Kingdom,Professional and managerial2,Inflow,Count,People (thousands),Estimate,None,11.0
9,64,NaN,1991.0,United Kingdom,Manual and clerical2,Inflow,Count,People (thousands),Estimate,None,11.0


In [15]:
new_table.count()

OBS             991
DATAMARKER       44
Year            948
Geography       984
Occupation      991
Flow            984
Measure Type    991
Unit            991
Tag             991
Revision         12
CI              991
dtype: int64

In [16]:
# new_table.to_csv('abc.csv')

In [17]:
# def user_perc1(x,y):
    
#     if str(x) == 'All persons':
#         return y
#     else:
#         return 'None'
    
# new_table['Revisions'] = new_table.apply(lambda row: user_perc1(row['Occupation'], row['Revisions']), axis = 1)

In [18]:
# new_table.head(5)

In [19]:
new_table = new_table[new_table['Year'].isnull() == False]

In [20]:
new_table = new_table[new_table['Year'] !=  0 ]

In [21]:
new_table.count()

OBS             948
DATAMARKER        1
Year            948
Geography       948
Occupation      948
Flow            948
Measure Type    948
Unit            948
Tag             948
Revision         11
CI              948
dtype: int64

In [22]:
new_table[DATAMARKER].unique()

array([nan, 'Statistically Significant Decrease'], dtype=object)

In [23]:
new_table[new_table['DATAMARKER'] != 'None']

,OBS,DATAMARKER,Year,Geography,Occupation,Flow,Measure Type,Unit,Tag,Revision,CI
7,329,NaN,1991.0,United Kingdom,All persons,Inflow,Count,People (thousands),Estimate,None,23.0
8,94,NaN,1991.0,United Kingdom,Professional and managerial2,Inflow,Count,People (thousands),Estimate,None,11.0
9,64,NaN,1991.0,United Kingdom,Manual and clerical2,Inflow,Count,People (thousands),Estimate,None,11.0
10,57,NaN,1991.0,United Kingdom,Students2,Inflow,Count,People (thousands),Estimate,None,10.0
11,58,NaN,1991.0,United Kingdom,Other adults3,Inflow,Count,People (thousands),Estimate,None,9.0
12,56,NaN,1991.0,United Kingdom,Children4,Inflow,Count,People (thousands),Estimate,None,10.0
13,268,NaN,1992.0,United Kingdom,All persons,Inflow,Count,People (thousands),Estimate,None,20.0
14,74,NaN,1992.0,United Kingdom,Professional and managerial2,Inflow,Count,People (thousands),Estimate,None,10.0
15,51,NaN,1992.0,United Kingdom,Manual and clerical2,Inflow,Count,People (thousands),Estimate,None,8.0
16,50,NaN,1992.0,United Kingdom,Students2,Inflow,Count,People (thousands),Estimate,None,8.0


In [24]:
# def user_perc6(x,y,z):
    
#     if (str(x) ==  ':') :
        
#         return str(y)
#     else:
#         return 
    
# new_table['temp1'] = new_table.apply(lambda row: user_perc6(row['CI'], row['Flow'], row['Revision']), axis = 1)

In [25]:
# def user_perc6(x,y,z):
    
#     if ((str(x) ==  ':') & ((str(y) == 'Balance')):     
        
#             return str(z)    
#     else:
#             return None
    
# new_table['temp1'] = new_table.apply(lambda row: user_perc6(row['CI'], row['Flow'], row['Revision']), axis = 1)

In [26]:
# def user_perc6(x,y,z):
    
#     if (str(x) ==  'Balance') & ((str(y) == '2001')) : 
# #                                  (str(y) == '2002') | 
# #                                  (str(y) == '2003') | 
# #                                  (str(y) == '2004') |
# #                                  (str(y) == '2005') |  
# #                                  (str(y) == '2006') | 
# #                                  (str(y) == '2007') | 
# #                                  (str(y) == '2008') |
# #                                  (str(y) == '2009') |  
# #                                  (str(y) == '2010') |  
# #                                  (str(y) == '2011')) :         
#             return str(z)    
#     else:
#             return None
    
# new_table['temp2'] = new_table.apply(lambda row: user_perc6(row['Flow'], row['Year'], row['Revision']), axis = 1)

In [27]:
# def user_perc7(x,y,z):
    
#     if ((str(x) ==  'All persons') & (str(y) == ':')) : 
                                         
#             return str(z)    
#     else:
#             return None
    
# new_table['temp2'] = new_table.apply(lambda row: user_perc6(row['Occupation'], row['CI'], row['Revision']), axis = 1)

In [28]:
# new_table.OBS.fillna('0', inplace = True)

In [29]:
# new_table.fillna('None', inplace = True)

In [30]:
new_table.head()

,OBS,DATAMARKER,Year,Geography,Occupation,Flow,Measure Type,Unit,Tag,Revision,CI
7,329,NaN,1991.0,United Kingdom,All persons,Inflow,Count,People (thousands),Estimate,None,23.0
8,94,NaN,1991.0,United Kingdom,Professional and managerial2,Inflow,Count,People (thousands),Estimate,None,11.0
9,64,NaN,1991.0,United Kingdom,Manual and clerical2,Inflow,Count,People (thousands),Estimate,None,11.0
10,57,NaN,1991.0,United Kingdom,Students2,Inflow,Count,People (thousands),Estimate,None,10.0
11,58,NaN,1991.0,United Kingdom,Other adults3,Inflow,Count,People (thousands),Estimate,None,9.0


In [31]:
def user_perc(x):
    
    if str(x) == ':':
        return '2011 Census Revision'
    else:
        return 'Original Estimate'
    
new_table['Revision'] = new_table.apply(lambda row: user_perc(row['CI']), axis = 1)

In [32]:
# def user_perc(x,y):
    
#     if str(x) == 'None':
#         return y
#     else:
#         return 0
    
# new_table['OBS'] = new_table.apply(lambda row: user_perc(row['DATAMARKER'], row['OBS']), axis = 1)

In [33]:
# new_table[new_table['OBS'] == 'z']

In [34]:
new_table.count()

OBS             948
DATAMARKER        1
Year            948
Geography       948
Occupation      948
Flow            948
Measure Type    948
Unit            948
Tag             948
Revision        948
CI              948
dtype: int64

In [35]:
new_table['Occupation'] = new_table['Occupation'].str.rstrip('234')

In [36]:
new_table['Revision'] = new_table['Revision'].str.rstrip('s1')

In [37]:
new_table.head()

,OBS,DATAMARKER,Year,Geography,Occupation,Flow,Measure Type,Unit,Tag,Revision,CI
7,329,NaN,1991.0,United Kingdom,All persons,Inflow,Count,People (thousands),Estimate,Original Estimate,23.0
8,94,NaN,1991.0,United Kingdom,Professional and managerial,Inflow,Count,People (thousands),Estimate,Original Estimate,11.0
9,64,NaN,1991.0,United Kingdom,Manual and clerical,Inflow,Count,People (thousands),Estimate,Original Estimate,11.0
10,57,NaN,1991.0,United Kingdom,Students,Inflow,Count,People (thousands),Estimate,Original Estimate,10.0
11,58,NaN,1991.0,United Kingdom,Other adults,Inflow,Count,People (thousands),Estimate,Original Estimate,9.0


In [38]:
new_table.count()

OBS             948
DATAMARKER        1
Year            948
Geography       948
Occupation      948
Flow            948
Measure Type    948
Unit            948
Tag             948
Revision        948
CI              948
dtype: int64

In [39]:
def user_perc(x):
    
    if str(x) == ':':
        return 0
    else:
        return x
    
new_table['CI'] = new_table.apply(lambda row: user_perc(row['CI']), axis = 1)

In [40]:
# new_table['Key'] = new_table['Year'] + '/' + new_table['Geography'] + '/' + new_table['Occupation'] + '/' + new_table['Flow'] + new_table['Revisions']

In [41]:
# Final_table = pd.merge(new_table, new_table[['Key','OBS','Tag']], how = 'left', left_on = 'Key', right_on = 'Key')

In [42]:
# Final_table  = Final_table[Final_table['Tag_x'] != Final_table['Tag_y']]

In [43]:
# Final_table.count()

In [44]:
# Final_table.drop_duplicates('Key', inplace = True)

In [45]:
# Final_table.count()

In [46]:
# Final_table

In [47]:
# Final_table['Occupation'] = Final_table['Occupation'] + ':' + Final_table['Revisions']

In [48]:
# Final_table['Occupation'] = Final_table['Occupation'].map(lambda cell:cell.replace(':None', ''))

In [49]:
# Final_table['OBS_x'] = Final_table['OBS_x'].astype(int)

In [50]:
# Final_table['OBS_y'] = pd.to_numeric(Final_table['OBS_y'])

In [51]:
# Final_table['OBS_y'].fillna(0, inplace = True)

In [52]:
# Final_table['OBS_y'] = Final_table['OBS_y'].astype(int)

In [53]:
# Final_table['Value'] = Final_table['OBS'].astype(str)

In [54]:
# Final_table['CI'] = Final_table['CI'].astype(str)

In [55]:
# Final_table['Value'] = Final_table['OBS_x'].astype(str) + ' ± '+ Final_table['OBS_y'].astype(str)

In [56]:
# Final_table

In [57]:
# Final_table['Year'] = Final_table['Year'].astype(str)

In [58]:
# Final_table.dtypes

In [59]:
# Final_table.Year.unique()

In [60]:
# Final_table = Final_table[Final_table['Year'] !=  'None' ]

In [61]:
# Final_table['Year'] = Final_table(int(hex(Year),16))

In [62]:
# Final_table['Year'] = pd.to_numeric(Final_table['Year'])

In [63]:
# Final_table['Year'] = Final_table['Year'].astype(int)

In [64]:
# Final_table.dtypes

In [65]:
# Final_table.tail(5)

In [66]:
new_table['Geography'] = new_table['Geography'].map(lambda cell:cell.replace('United Kingdom', 'K02000001'))

In [67]:
new_table['Geography'] = new_table['Geography'].map(lambda cell:cell.replace('England and Wales', 'K04000001'))

In [68]:
new_table.dtypes

OBS             object
DATAMARKER      object
Year            object
Geography       object
Occupation      object
Flow            object
Measure Type    object
Unit            object
Tag             object
Revision        object
CI              object
dtype: object

In [69]:
new_table[new_table['OBS'] != '']

,OBS,DATAMARKER,Year,Geography,Occupation,Flow,Measure Type,Unit,Tag,Revision,CI
7,329,NaN,1991.0,K02000001,All persons,Inflow,Count,People (thousands),Estimate,Original Estimate,23.0
8,94,NaN,1991.0,K02000001,Professional and managerial,Inflow,Count,People (thousands),Estimate,Original Estimate,11.0
9,64,NaN,1991.0,K02000001,Manual and clerical,Inflow,Count,People (thousands),Estimate,Original Estimate,11.0
10,57,NaN,1991.0,K02000001,Students,Inflow,Count,People (thousands),Estimate,Original Estimate,10.0
11,58,NaN,1991.0,K02000001,Other adults,Inflow,Count,People (thousands),Estimate,Original Estimate,9.0
12,56,NaN,1991.0,K02000001,Children,Inflow,Count,People (thousands),Estimate,Original Estimate,10.0
13,268,NaN,1992.0,K02000001,All persons,Inflow,Count,People (thousands),Estimate,Original Estimate,20.0
14,74,NaN,1992.0,K02000001,Professional and managerial,Inflow,Count,People (thousands),Estimate,Original Estimate,10.0
15,51,NaN,1992.0,K02000001,Manual and clerical,Inflow,Count,People (thousands),Estimate,Original Estimate,8.0
16,50,NaN,1992.0,K02000001,Students,Inflow,Count,People (thousands),Estimate,Original Estimate,8.0


In [70]:
new_table['Year'] = new_table['Year'].astype(str)

In [71]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [72]:
new_table['Year'] = new_table['Year'].astype(int)

In [73]:
new_table['Value'] = new_table['OBS'].astype(str)

In [74]:
new_table['Value'] = pd.to_numeric(new_table['Value'], errors='coerce').fillna(0)

In [75]:
new_table['Value'] = new_table['Value'].astype(int)

In [76]:
new_table['CI'] = new_table['CI'].astype(str)

In [77]:
new_table['CI'] = pd.to_numeric(new_table['CI'], errors='coerce').fillna(0)

In [78]:
new_table['CI'] = new_table['CI'].astype(int)

In [79]:
new_table = new_table[new_table['Year'] !=  0 ]

In [80]:
new_table.dtypes

OBS             object
DATAMARKER      object
Year             int64
Geography       object
Occupation      object
Flow            object
Measure Type    object
Unit            object
Tag             object
Revision        object
CI               int64
Value            int64
dtype: object

In [81]:
new_table = new_table[['Geography','Year','Occupation','Flow','Measure Type','Value', 'CI','Unit','Revision']]

In [82]:
new_table.head(5)

,Geography,Year,Occupation,Flow,Measure Type,Value,CI,Unit,Revision
7,K02000001,1991,All persons,Inflow,Count,329,23,People (thousands),Original Estimate
8,K02000001,1991,Professional and managerial,Inflow,Count,94,11,People (thousands),Original Estimate
9,K02000001,1991,Manual and clerical,Inflow,Count,64,11,People (thousands),Original Estimate
10,K02000001,1991,Students,Inflow,Count,57,10,People (thousands),Original Estimate
11,K02000001,1991,Other adults,Inflow,Count,58,9,People (thousands),Original Estimate


In [83]:
from pathlib import Path
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

new_table.drop_duplicates().to_csv(destinationFolder / ('observations.csv'), index = False)

In [84]:
from gssutils.metadata import THEME
scraper.dataset.theme = THEME['population']
scraper.dataset.family = 'migration'

with open(destinationFolder / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())

In [85]:
new_table.count()

Geography       947
Year            947
Occupation      947
Flow            947
Measure Type    947
Value           947
CI              947
Unit            947
Revision        947
dtype: int64